In [1]:
pip install pandas scikit-learn fairlearn


Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from fairlearn.metrics import MetricFrame, selection_rate, true_positive_rate, true_negative_rate, false_positive_rate, false_negative_rate

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"]

data = pd.read_csv(url, names=column_names, sep=',\s', engine='python')

# Preprocess the dataset
# Drop rows with missing values
data = data.replace('?', pd.NA).dropna()

# Encode categorical features
categorical_columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country", "income"]
data[categorical_columns] = data[categorical_columns].apply(LabelEncoder().fit_transform)


# Define features and target
X = data.drop("income", axis=1)
y = data["income"]
sensitive_feature = data["sex"]  # Use sex as the sensitive feature

# Split the dataset
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(X, y, sensitive_feature, test_size=0.25, random_state=42)



In [17]:
# Train a logistic regression model with fairness constraints
estimator = LogisticRegression(max_iter=1000)
constraint = DemographicParity()  # You can also use EqualizedOdds() for a different fairness constraint

mitigator = ExponentiatedGradient(estimator, constraint)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_train)

# Predict using the mitigated model
y_pred_mitigated = mitigator.predict(X_test)

# Evaluate the mitigated model
accuracy_mitigated = accuracy_score(y_test, y_pred_mitigated)
print(f"Mitigated Model accuracy: {accuracy_mitigated:.2f}")

# Define a dictionary of metrics for the mitigated model
metrics_mitigated = {
    'selection_rate': selection_rate,
    'true_positive_rate': true_positive_rate,
    'true_negative_rate': true_negative_rate,
    'false_positive_rate': false_positive_rate,
    'false_negative_rate': false_negative_rate
}

# Create a MetricFrame to evaluate metrics by sensitive feature for the mitigated model
metric_frame_mitigated = MetricFrame(
    metrics=metrics_mitigated,
    y_true=y_test,
    y_pred=y_pred_mitigated,
    sensitive_features=sensitive_test  # Already mapped to 'female' and 'male'
)

# Print the overall metrics for the mitigated model
print("Overall metrics for mitigated model:")
print(metric_frame_mitigated.overall)

# Mapping for display
sex_display_mapping = {0: 'female', 1: 'male'}

# Print the metrics by group for the mitigated model, with human-readable labels
print("Metrics by group for mitigated model:")
metrics_by_group_mitigated = metric_frame_mitigated.by_group.rename(index=sex_display_mapping)
print(metrics_by_group_mitigated)

Mitigated Model accuracy: 0.78
Overall metrics for mitigated model:
selection_rate         0.086063
true_positive_rate     0.236828
true_negative_rate     0.963970
false_positive_rate    0.036030
false_negative_rate    0.763172
dtype: float64
Metrics by group for mitigated model:
        selection_rate  true_positive_rate  true_negative_rate  \
sex                                                              
female        0.082286            0.336957            0.949795   
male          0.087899            0.219588            0.972918   

        false_positive_rate  false_negative_rate  
sex                                               
female             0.050205             0.663043  
male               0.027082             0.780412  


In [11]:
# Calculate Disparate Impact
sr_by_group = metric_frame.by_group['selection_rate']
disparate_impact = sr_by_group.min() / sr_by_group.max()
bias_percentage = (1 - disparate_impact) * 100

print(f"Disparate Impact: {disparate_impact:.2f}")
print(f"Bias Percentage: {bias_percentage:.2f}%")


Disparate Impact: 0.58
Bias Percentage: 42.23%


In [18]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds

# Train a logistic regression model with fairness constraints
estimator = LogisticRegression(max_iter=1000)
constraint = DemographicParity()  # You can also use EqualizedOdds() for a different fairness constraint

mitigator = ExponentiatedGradient(estimator, constraint)
mitigator.fit(X_train, y_train, sensitive_features=sensitive_train)

# Predict using the mitigated model
y_pred_mitigated = mitigator.predict(X_test)

# Evaluate the mitigated model
accuracy_mitigated = accuracy_score(y_test, y_pred_mitigated)
print(f"Mitigated Model accuracy: {accuracy_mitigated:.2f}")

# Define a dictionary of metrics for the mitigated model
metrics_mitigated = {
    'selection_rate': selection_rate,
    'true_positive_rate': true_positive_rate,
    'true_negative_rate': true_negative_rate,
    'false_positive_rate': false_positive_rate,
    'false_negative_rate': false_negative_rate
}

# Create a MetricFrame to evaluate metrics by sensitive feature for the mitigated model
metric_frame_mitigated = MetricFrame(
    metrics=metrics_mitigated,
    y_true=y_test,
    y_pred=y_pred_mitigated,
    sensitive_features=sensitive_test
)

# Print the overall metrics for the mitigated model
print("Overall metrics for mitigated model:")
print(metric_frame_mitigated.overall)

# Mapping for display
sex_display_mapping = {0: 'female', 1: 'male'}

# Print the metrics by group for the mitigated model, with human-readable labels
print("Metrics by group for mitigated model:")
metrics_by_group_mitigated = metric_frame_mitigated.by_group.rename(index=sex_display_mapping)
print(metrics_by_group_mitigated)


Mitigated Model accuracy: 0.78
Overall metrics for mitigated model:
selection_rate         0.083676
true_positive_rate     0.230974
true_negative_rate     0.965207
false_positive_rate    0.034793
false_negative_rate    0.769026
dtype: float64
Metrics by group for mitigated model:
        selection_rate  true_positive_rate  true_negative_rate  \
sex                                                              
female        0.075801            0.307971            0.953446   
male          0.087505            0.217717            0.972630   

        false_positive_rate  false_negative_rate  
sex                                               
female             0.046554             0.692029  
male               0.027370             0.782283  


In [6]:
# Calculate Disparate Impact for the mitigated model
sr_by_group_mitigated = metric_frame_mitigated.by_group['selection_rate']
disparate_impact_mitigated = sr_by_group_mitigated.min() / sr_by_group_mitigated.max()
bias_percentage_mitigated = (1 - disparate_impact_mitigated) * 100

print(f"Mitigated Disparate Impact: {disparate_impact_mitigated:.2f}")
print(f"Mitigated Bias Percentage: {bias_percentage_mitigated:.2f}%")


Mitigated Disparate Impact: 0.87
Mitigated Bias Percentage: 12.72%


In [19]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best estimator
best_estimator = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")

# Train the best estimator with fairness constraints
mitigator = ExponentiatedGradient(best_estimator, DemographicParity())
mitigator.fit(X_train, y_train, sensitive_features=sensitive_train)

# Predict using the mitigated model
y_pred_mitigated = mitigator.predict(X_test)

# Evaluate the mitigated model
accuracy_mitigated = accuracy_score(y_test, y_pred_mitigated)
print(f"Mitigated Model accuracy after tuning: {accuracy_mitigated:.2f}")

# Define a dictionary of metrics for the mitigated model
metrics_mitigated = {
    'selection_rate': selection_rate,
    'true_positive_rate': true_positive_rate,
    'true_negative_rate': true_negative_rate,
    'false_positive_rate': false_positive_rate,
    'false_negative_rate': false_negative_rate
}

# Create a MetricFrame to evaluate metrics by sensitive feature for the mitigated model
metric_frame_mitigated = MetricFrame(
    metrics=metrics_mitigated,
    y_true=y_test,
    y_pred=y_pred_mitigated,
    sensitive_features=sensitive_test
)

# Print the overall metrics for the mitigated model
print("Overall metrics for mitigated model after tuning:")
print(metric_frame_mitigated.overall)

# Mapping for display
sex_display_mapping = {0: 'female', 1: 'male'}

# Print the metrics by group for the mitigated model, with human-readable labels
print("Metrics by group for mitigated model:")
metrics_by_group_mitigated = metric_frame_mitigated.by_group.rename(index=sex_display_mapping)
print(metrics_by_group_mitigated)

# Calculate Disparate Impact for the mitigated model
sr_by_group_mitigated = metric_frame_mitigated.by_group['selection_rate']
disparate_impact_mitigated = sr_by_group_mitigated.min() / sr_by_group_mitigated.max()
bias_percentage_mitigated = (1 - disparate_impact_mitigated) * 100

print(f"Mitigated Disparate Impact after tuning: {disparate_impact_mitigated:.2f}")
print(f"Mitigated Bias Percentage after tuning: {bias_percentage_mitigated:.2f}%")


Best parameters: {'C': 1, 'solver': 'liblinear'}
Mitigated Model accuracy after tuning: 0.79
Overall metrics for mitigated model after tuning:
selection_rate         0.070548
true_positive_rate     0.213944
true_negative_rate     0.977040
false_positive_rate    0.022960
false_negative_rate    0.786056
dtype: float64
Metrics by group for mitigated model:
        selection_rate  true_positive_rate  true_negative_rate  \
sex                                                              
female        0.064045            0.278986            0.963031   
male          0.073709            0.202745            0.985883   

        false_positive_rate  false_negative_rate  
sex                                               
female             0.036969             0.721014  
male               0.014117             0.797255  
Mitigated Disparate Impact after tuning: 0.87
Mitigated Bias Percentage after tuning: 13.11%


In [11]:
# Evaluate additional metrics
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred_mitigated)
recall = recall_score(y_test, y_pred_mitigated)
f1 = f1_score(y_test, y_pred_mitigated)

print(f"Mitigated Model Precision: {precision:.2f}")
print(f"Mitigated Model Recall: {recall:.2f}")
print(f"Mitigated Model F1 Score: {f1:.2f}")


Mitigated Model Precision: 0.74
Mitigated Model Recall: 0.21
Mitigated Model F1 Score: 0.33


In [20]:
# Train a logistic regression model with EqualizedOdds constraint
mitigator_equalized_odds = ExponentiatedGradient(best_estimator, EqualizedOdds())
mitigator_equalized_odds.fit(X_train, y_train, sensitive_features=sensitive_train)

# Predict using the mitigated model with EqualizedOdds
y_pred_equalized_odds = mitigator_equalized_odds.predict(X_test)

# Evaluate the model with EqualizedOdds constraint
accuracy_equalized_odds = accuracy_score(y_test, y_pred_equalized_odds)
print(f"Equalized Odds Model accuracy: {accuracy_equalized_odds:.2f}")

# Create a MetricFrame to evaluate metrics by sensitive feature for the EqualizedOdds model
metric_frame_equalized_odds = MetricFrame(
    metrics=metrics_mitigated,
    y_true=y_test,
    y_pred=y_pred_equalized_odds,
    sensitive_features=sensitive_test
)

# Print the overall metrics for the EqualizedOdds model
print("Overall metrics for EqualizedOdds model:")
print(metric_frame_equalized_odds.overall)

# Mapping for display
sex_display_mapping = {0: 'female', 1: 'male'}

# Print the metrics by group for the mitigated model, with human-readable labels
print("Metrics by group for mitigated model:")
metrics_by_group_mitigated = metric_frame_mitigated.by_group.rename(index=sex_display_mapping)
print(metrics_by_group_mitigated)


Equalized Odds Model accuracy: 0.79
Overall metrics for EqualizedOdds model:
selection_rate         0.087522
true_positive_rate     0.263970
true_negative_rate     0.971035
false_positive_rate    0.028965
false_negative_rate    0.736030
dtype: float64
Metrics by group for mitigated model:
        selection_rate  true_positive_rate  true_negative_rate  \
sex                                                              
female        0.064045            0.278986            0.963031   
male          0.073709            0.202745            0.985883   

        false_positive_rate  false_negative_rate  
sex                                               
female             0.036969             0.721014  
male               0.014117             0.797255  


In [21]:
from fairlearn.metrics import MetricFrame, selection_rate

# Define a dictionary of metrics for the mitigated model
metrics_mitigated = {
    'selection_rate': selection_rate,
    'true_positive_rate': true_positive_rate,
    'true_negative_rate': true_negative_rate,
    'false_positive_rate': false_positive_rate,
    'false_negative_rate': false_negative_rate
}

# Train a logistic regression model with EqualizedOdds constraint
mitigator_equalized_odds = ExponentiatedGradient(best_estimator, EqualizedOdds())
mitigator_equalized_odds.fit(X_train, y_train, sensitive_features=sensitive_train)

# Predict using the mitigated model with EqualizedOdds
y_pred_equalized_odds = mitigator_equalized_odds.predict(X_test)

# Evaluate the model with EqualizedOdds constraint
accuracy_equalized_odds = accuracy_score(y_test, y_pred_equalized_odds)
print(f"Equalized Odds Model accuracy: {accuracy_equalized_odds:.2f}")

# Create a MetricFrame to evaluate metrics by sensitive feature for the EqualizedOdds model
metric_frame_equalized_odds = MetricFrame(
    metrics=metrics_mitigated,
    y_true=y_test,
    y_pred=y_pred_equalized_odds,
    sensitive_features=sensitive_test
)

# Print the overall metrics for the EqualizedOdds model
print("Overall metrics for EqualizedOdds model:")
print(metric_frame_equalized_odds.overall)

# Mapping for display
sex_display_mapping = {0: 'female', 1: 'male'}

# Print the metrics by group for the mitigated model, with human-readable labels
print("Metrics by group for mitigated model:")
metrics_by_group_mitigated = metric_frame_mitigated.by_group.rename(index=sex_display_mapping)
print(metrics_by_group_mitigated)

# Calculate and Print Bias Metrics for the EqualizedOdds Model
sr_by_group_equalized_odds = metric_frame_equalized_odds.by_group['selection_rate']
disparate_impact_equalized_odds = sr_by_group_equalized_odds.min() / sr_by_group_equalized_odds.max()
bias_percentage_equalized_odds = (1 - disparate_impact_equalized_odds) * 100

print(f"Equalized Odds Model Disparate Impact: {disparate_impact_equalized_odds:.2f}")
print(f"Equalized Odds Model Bias Percentage: {bias_percentage_equalized_odds:.2f}%")


Equalized Odds Model accuracy: 0.79
Overall metrics for EqualizedOdds model:
selection_rate         0.087522
true_positive_rate     0.263970
true_negative_rate     0.971035
false_positive_rate    0.028965
false_negative_rate    0.736030
dtype: float64
Metrics by group for mitigated model:
        selection_rate  true_positive_rate  true_negative_rate  \
sex                                                              
female        0.064045            0.278986            0.963031   
male          0.073709            0.202745            0.985883   

        false_positive_rate  false_negative_rate  
sex                                               
female             0.036969             0.721014  
male               0.014117             0.797255  
Equalized Odds Model Disparate Impact: 0.57
Equalized Odds Model Bias Percentage: 43.11%


In [ ]:
# Dynamic patch for np.PINF in fairlearn
import fairlearn.reductions._exponentiated_gradient.exponentiated_gradient as eg
if hasattr(np, 'PINF'):
    np.PINF = np.inf  # Just in case, though it should already be removed
eg.np.PINF = np.inf
